In [1]:
import pypsa
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
n=pypsa.Network()

#Snapshots
n.set_snapshots(value=pd.date_range(freq="h", start="2013-01-01", end="2014-01-01", inclusive='left'))
n.snapshot_weightings[:] *= 8760.0 / n.snapshot_weightings.sum()
Nyears = n.snapshot_weightings.objective.sum() / 8760.0

# # Load yaml files
import yaml

with open(r'config.yaml') as file:
        config = yaml.load(file, Loader=yaml.FullLoader)
 
        print(config)

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:374: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:374: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:374: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:374: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:374: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages

{'version': '0.0.2', 'tutorial': True, 'logging': {'level': 'INFO', 'format': '%(levelname)s:%(name)s:%(message)s'}, 'scenario': {'simpl': [''], 'll': ['copt'], 'clusters': [6], 'opts': ['Co2L-4H']}, 'countries': ['NG'], 'summary_dir': 'results', 'snapshots': {'start': '2013-03-1', 'end': '2013-03-7', 'inclusive': 'left'}, 'enable': {'retrieve_databundle': False, 'download_osm_data': False, 'build_cutout': False, 'build_natura_raster': False}, 'crs': {'geo_crs': 'EPSG:4326', 'distance_crs': 'EPSG:3857', 'area_crs': 'ESRI:54009'}, 'retrieve_databundle': {'show_progress': False}, 'augmented_line_connection': {'add_to_snakefile': False, 'connectivity_upgrade': 2, 'new_line_type': ['HVAC'], 'min_expansion': 1, 'min_DC_length': 600}, 'cluster_options': {'alternative_clustering': False, 'distribute_cluster': ['load'], 'out_logging': True, 'aggregation_strategies': {'generators': {'p_nom': 'sum', 'p_nom_max': 'sum', 'p_nom_min': 'sum', 'p_min_pu': 'mean', 'marginal_cost': 'mean', 'committable

In [2]:
#I create the bus 

n.madd("Bus", ["onebus"], x=0.1, y=0.2, carrier="AC", v_nom=20)



Index(['onebus'], dtype='object')

In [3]:
costs=pd.read_csv(r'C:\Users\denis\OneDrive\Desktop\Mini grids\pypsa-distribution\costs.csv')

In [4]:
tech_costs=pd.read_csv(r'C:\Users\denis\OneDrive\Desktop\Mini grids\pypsa-distribution\costs.csv')

idx = pd.IndexSlice

def calculate_annuity(n, r):
    """
    Calculate the annuity factor for an asset with lifetime n years and
    discount rate of r, e.g. annuity(20, 0.05) * 20 = 1.6
    """
    if isinstance(r, pd.Series):
        return pd.Series(1 / n, index=r.index).where(
            r == 0, r / (1.0 - 1.0 / (1.0 + r) ** n)
        )
    elif r > 0:
        return r / (1.0 - 1.0 / (1.0 + r) ** n)
    else:
        return 1 / n

In [5]:
def _add_missing_carriers_from_costs(n, costs, carriers):
    missing_carriers = pd.Index(carriers).difference(n.carriers.index)
    if missing_carriers.empty:
        return

    emissions_cols = (
        costs.columns.to_series().loc[lambda s: s.str.endswith("_emissions")].values
    )
    suptechs = missing_carriers.str.split("-").str[0]
    emissions = costs.loc[suptechs, emissions_cols].fillna(0.0)
    emissions.index = missing_carriers

In [6]:
tech_costs='C://Users//denis//OneDrive//Desktop//Mini grids//pypsa-distribution//costs.csv'

In [7]:
def load_costs(tech_costs, config, elec_config, Nyears=1):
    """
    set all asset costs and other parameters
    """
    costs = pd.read_csv(tech_costs, index_col=list(range(3))).sort_index()

    # correct units to MW and EUR
    costs.loc[costs.unit.str.contains("/kW"), "value"] *= 1e3
    costs.loc[costs.unit.str.contains("USD"), "value"] *= config["USD2013_to_EUR2013"]

    costs = (
        costs.loc[idx[:, config["year"], :], "value"]
        .unstack(level=2)
        .groupby("technology")
        .sum(min_count=1)
    )

    costs = costs.fillna(
        {
            "CO2 intensity": 0,
            "FOM": 0,
            "VOM": 0,
            "discount rate": config["discountrate"],
            "efficiency": 1,
            "fuel": 0,
            "investment": 0,
            "lifetime": 25,
        }
    )

    costs["capital_cost"] = (
        (
            calculate_annuity(costs["lifetime"], costs["discount rate"])
            + costs["FOM"] / 100.0
        )
        * costs["investment"]
        * Nyears
    )

    costs.at["OCGT", "fuel"] = costs.at["gas", "fuel"]
    costs.at["CCGT", "fuel"] = costs.at["gas", "fuel"]

    costs["marginal_cost"] = costs["VOM"] + costs["fuel"] / costs["efficiency"]

    costs = costs.rename(columns={"CO2 intensity": "co2_emissions"})

    costs.at["OCGT", "co2_emissions"] = costs.at["gas", "co2_emissions"]
    costs.at["CCGT", "co2_emissions"] = costs.at["gas", "co2_emissions"]

    costs.at["solar", "capital_cost"] = 0.5 * (
        costs.at["solar-rooftop", "capital_cost"]
        + costs.at["solar-utility", "capital_cost"]
    )

    def costs_for_storage(store, link1, link2=None, max_hours=1.0):
        capital_cost = link1["capital_cost"] + max_hours * store["capital_cost"]
        if link2 is not None:
            capital_cost += link2["capital_cost"]
        return pd.Series(
            dict(capital_cost=capital_cost, marginal_cost=0.0, co2_emissions=0.0)
        )

    max_hours = elec_config["max_hours"]
    costs.loc["battery"] = costs_for_storage(
        costs.loc["battery storage"],
        costs.loc["battery inverter"],
        max_hours=max_hours["battery"],
    )
    costs.loc["H2"] = costs_for_storage(
        costs.loc["hydrogen storage"],
        costs.loc["fuel cell"],
        costs.loc["electrolysis"],
        max_hours=max_hours["H2"],
    )

    for attr in ("marginal_cost", "capital_cost"):
        overwrites = config.get(attr)
        if overwrites is not None:
            overwrites = pd.Series(overwrites)
            costs.loc[overwrites.index, attr] = overwrites

    return costs

In [8]:
costs = load_costs(
    tech_costs,
    config["costs"],
    config["electricity"],
    Nyears,
    )

In [9]:
def attach_wind_and_solar(n, costs, tech_modelling, extendable_carriers):

    _add_missing_carriers_from_costs(n, costs, tech_modelling)

    for tech in tech_modelling:
       
        with xr.open_dataset(f"renewable_profiles/profile_{tech}.nc") as ds:
            
            if ds.indexes["bus"].empty:
                continue   

            suptech = tech.split("-", 2)[0]
          
            n.madd(
            "Generator",
            ds.indexes["bus"],
            " " + tech,
            bus="onebus",
            carrier=tech,
            p_nom_extendable=tech in extendable_carriers["Generator"],
            p_nom_max=ds["p_nom_max"].to_pandas(), #guardare il config 
            weight=ds["weight"].to_pandas(),
            marginal_cost=costs.at[suptech, "marginal_cost"],
            capital_cost=costs.at[tech, "capital_cost"],
            efficiency=costs.at[suptech, "efficiency"],
            p_set=ds["profile"].transpose("time", "bus").to_pandas().reindex(n.snapshots),
            p_max_pu=ds["profile"].transpose("time", "bus").to_pandas().reindex(n.snapshots),
            )


In [10]:
attach_wind_and_solar(
    n,
    costs,
    config["tech_modelling"]["general_vre"],
    config["electricity"]["extendable_carriers"],
    )

In [12]:
def attach_storageunits(n, costs,technologies, extendable_carriers ):

    buses_i = n.buses.index

    for tech in technologies:
        
        n.madd(
            "StorageUnit",
            buses_i, 
            " " + tech, 
            bus=["onebus"],
            carrier=tech,
            p_nom_extendable=True,
            capital_cost=100,
            marginal_cost=10,
            cyclic_state_of_charge=True
            
        )
    

   # capital_cost=costs.at[tech, "capital_cost"],
            # marginal_cost=costs.at[tech, "marginal_cost"],
            # # efficiency_store=costs.at[lookup_store[carrier], "efficiency"],
            # # efficiency_dispatch=costs.at[lookup_dispatch[carrier], "efficiency"],
            # # max_hours=max_hours[carrier],
            # cyclic_state_of_charge=True,
   

In [13]:
attach_storageunits(n, 
                    costs,
                    config["tech_modelling"]["storage_techs"],
                    config["electricity"]["extendable_carriers"],
                    )

In [14]:
n.storage_units

,bus,carrier,p_nom_extendable,capital_cost,marginal_cost,cyclic_state_of_charge,control,type,p_nom,p_nom_min,...,state_of_charge_initial,state_of_charge_initial_per_period,state_of_charge_set,cyclic_state_of_charge_per_period,max_hours,efficiency_store,efficiency_dispatch,standing_loss,inflow,p_nom_opt
StorageUnit,,,,,,,,,,,,,,,,,,,,,
onebus lithium,onebus,lithium,True,100.0,10.0,True,PQ,,0.0,0.0,...,0.0,False,NaN,True,1.0,1.0,1.0,0.0,0.0,0.0
onebus lead acid,onebus,lead acid,True,100.0,10.0,True,PQ,,0.0,0.0,...,0.0,False,NaN,True,1.0,1.0,1.0,0.0,0.0,0.0


In [15]:

#I create the load file
n_load=35 #number of loads
import numpy as np
p_set=np.random.rand(len(n.snapshots),(n_load))
array=p_set*100

In [16]:
import pandas as pd
df=pd.DataFrame(array)

filepath = 'my_file.xlsx'

df.to_excel(filepath, index=False)



In [17]:
load_df=pd.read_excel(r'C:\Users\denis\OneDrive\Desktop\Mini grids\pypsa-distribution\my_file.xlsx')


In [18]:

load_df=load_df.set_index([n.snapshots])
load_df.index.names=['time']
print(load_df)

                            0          1          2          3          4   \
time                                                                         
2013-01-01 00:00:00   2.170852  63.591388  16.951690  87.776400  44.513135   
2013-01-01 01:00:00  34.703151  87.516516  40.883350  67.300715  59.426721   
2013-01-01 02:00:00  95.825199  23.623355  89.273845  54.843893  90.623554   
2013-01-01 03:00:00  59.111954  66.752075  89.096991  20.211627  55.120140   
2013-01-01 04:00:00  13.054797  23.878825  77.789243  64.921814  18.691070   
...                        ...        ...        ...        ...        ...   
2013-12-31 19:00:00  70.627800  98.380225  76.616850  91.196139  32.483195   
2013-12-31 20:00:00  15.584477   4.606524  15.914601  53.358315  80.129569   
2013-12-31 21:00:00  46.652334  40.967016  71.698989  66.023419  66.905923   
2013-12-31 22:00:00  10.565089   2.191981  29.163679  22.165310  27.502796   
2013-12-31 23:00:00  49.062858  79.465493  93.942902  13.500895 

In [19]:
index=pd.Index( list(range(n_load)))
type(index)

pandas.core.indexes.numeric.Int64Index

In [20]:
def attach_load(n, load_paths, index, load_df):
    
    n.madd("Load", index ,bus=["onebus"], carrier="AC", p_set=load_df)




In [21]:
load_paths=r'C:\Users\denis\OneDrive\Desktop\Mini grids\pypsa-distribution\my_file.xlsx'

In [22]:
attach_load(n, load_paths, index, load_df)

In [23]:
n.loads

,bus,carrier,type,p_set,q_set,sign
Load,,,,,,
0,onebus,AC,,0.0,0.0,-1.0
1,onebus,AC,,0.0,0.0,-1.0
2,onebus,AC,,0.0,0.0,-1.0
3,onebus,AC,,0.0,0.0,-1.0
4,onebus,AC,,0.0,0.0,-1.0
5,onebus,AC,,0.0,0.0,-1.0
6,onebus,AC,,0.0,0.0,-1.0
7,onebus,AC,,0.0,0.0,-1.0
8,onebus,AC,,0.0,0.0,-1.0


In [24]:
print(n)

PyPSA Network
Components:
 - Bus: 1
 - Generator: 105
 - Load: 35
 - StorageUnit: 2
Snapshots: 8760


In [25]:
# Optimization
from pypsa.linopf import ilopf

solver_name="gurobi"

n.lopf(n.snapshots, solver_name=solver_name, pyomo=False)

INFO:pypsa.linopf:Prepare linear problem
INFO:pypsa.linopf:Total preparation time: 7.54s
INFO:pypsa.linopf:Solve linear problem using Gurobi solver


Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-07
Read LP format model from file C:\Users\denis\AppData\Local\Temp\pypsa-problem-qy67qgaq.lp
Reading time = 4.36 seconds
obj: 1971000 rows, 972468 columns, 3032023 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1971000 rows, 972468 columns and 3032023 nonzeros
Model fingerprint: 0x364effc8
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e-02, 1e+05]
  Bounds range     [4e+00, 2e+05]
  RHS range        [1e+03, 2e+03]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 1882337 rows and 909978 columns
Presolve time: 1.73s
Presolved: 88663 rows, 62490 columns, 239709 nonzeros

Ordering time: 0.04s

Barrier statistics:
 Dense cols : 54
 AA' NZ     : 1.833e+05
 Factor NZ  : 4.842e+05 (roughly 26 MB of memory)
 Factor

INFO:pypsa.linopf:Optimization successful. Objective value: 3.09e+10


('ok', 'optimal')